# Battery Data Tool - Button Functions Implementation

This notebook demonstrates the core functionality of each button in the Battery Data Tool.

## Table of Contents
1. [Setup & Imports](#setup)
2. [Individual Cycle Analysis](#indiv-cyc)
3. [Linked Cycle Analysis](#link-cyc)
4. [Step Profile Analysis](#step)
5. [Rate Capability Analysis](#rate)
6. [Charge Profile Analysis](#chg)
7. [Discharge Profile Analysis](#dchg)
8. [Continue Analysis (OCV/CCV)](#continue)
9. [DCIR Analysis](#dcir)

## 1. Setup & Imports <a id='setup'></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Configuration
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

# Color palette for graphs
COLORS = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
          '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

# Mock helper functions (replace with actual implementations)
def check_cycler(path):
    """Check if data is from PNE (True) or TOYO (False) cycler"""
    return 'PNE' in str(path) or 'pne' in str(path)

def progress(folder_cnt, folder_max, chnl_cnt, chnl_max, cyc_cnt, cyc_max):
    """Calculate progress percentage"""
    return ((folder_cnt + (chnl_cnt + cyc_cnt/cyc_max - 1)/chnl_max - 1)/folder_max) * 100

print("✅ Setup complete")

## 2. Individual Cycle Analysis <a id='indiv-cyc'></a>

**Purpose**: Analyze individual cycle data with separate visualization per channel

**Features**:
- 6-panel visualization (capacity, efficiency, DCIR)
- TOYO/PNE cycler support
- Optional DCIR analysis with SOC70 calculations
- Per-channel tab generation

In [ ]:
def indiv_cyc_analysis(data_folders, settings):
    """
    Individual Cycle Analysis Function
    
    Parameters:
        data_folders: list of folder paths containing cycle data
        settings: dict with 'firstCrate', 'mincapacity', 'xscale', 'ylimithigh', 'ylimitlow', 'irscale'
    
    Returns:
        dict with 'figures', 'data', 'export'
    """
    results = {'figures': [], 'data': [], 'export': pd.DataFrame()}
    
    for folder_idx, cyclefolder in enumerate(data_folders):
        if not os.path.exists(cyclefolder):
            continue
            
        # Get subfolders (channels)
        subfolders = [f.path for f in os.scandir(cyclefolder) if f.is_dir()]
        
        for chnl_idx, channel_folder in enumerate(subfolders):
            # Create 6-panel figure
            fig, axes = plt.subplots(2, 3, figsize=(14, 8))
            ax1, ax2, ax3, ax4, ax5, ax6 = axes.flatten()
            
            # Determine cycler type and load data
            is_pne = check_cycler(cyclefolder)
            
            # Mock data loading (replace with actual function)
            cycle_data = {
                'capacity': settings['mincapacity'],
                'dchg_capacity': np.random.rand(100) * settings['mincapacity'],
                'chg_capacity': np.random.rand(100) * settings['mincapacity'],
                'efficiency': np.random.rand(100) * 100,
                'dcir': np.random.rand(100) * 50,
                'rest_voltage': np.random.rand(100) * 0.5 + 3.5
            }
            
            # Plot 6 panels
            cycles = np.arange(len(cycle_data['dchg_capacity']))
            
            # Panel 1: Discharge Capacity
            ax1.plot(cycles, cycle_data['dchg_capacity'], 'o-', label='Dchg Cap')
            ax1.set_xlabel('Cycle')
            ax1.set_ylabel('Capacity (mAh)')
            ax1.legend(loc='lower left')
            ax1.grid(True, alpha=0.3)
            
            # Panel 2: Charge Capacity
            ax2.plot(cycles, cycle_data['chg_capacity'], 'o-', label='Chg Cap', color='orange')
            ax2.set_xlabel('Cycle')
            ax2.set_ylabel('Capacity (mAh)')
            ax2.legend(loc='lower right')
            ax2.grid(True, alpha=0.3)
            
            # Panel 3: Efficiency
            ax3.plot(cycles, cycle_data['efficiency'], 'o-', label='Efficiency', color='green')
            ax3.set_xlabel('Cycle')
            ax3.set_ylabel('Efficiency (%)')
            ax3.legend(loc='upper right')
            ax3.grid(True, alpha=0.3)
            
            # Panel 4: DCIR
            ax4.plot(cycles, cycle_data['dcir'], 'o-', label='DCIR', color='red')
            ax4.set_xlabel('Cycle')
            ax4.set_ylabel('DCIR (mΩ)')
            ax4.legend(loc='upper right')
            ax4.grid(True, alpha=0.3)
            
            # Panel 5: Rest Voltage
            ax5.plot(cycles, cycle_data['rest_voltage'], 'o-', label='Rest V', color='purple')
            ax5.set_xlabel('Cycle')
            ax5.set_ylabel('Voltage (V)')
            ax5.legend(loc='upper right')
            ax5.grid(True, alpha=0.3)
            
            # Panel 6: Temperature (placeholder)
            ax6.plot(cycles, np.random.rand(100) * 10 + 25, 'o-', label='Temp', color='brown')
            ax6.set_xlabel('Cycle')
            ax6.set_ylabel('Temperature (°C)')
            ax6.legend(loc='lower right')
            ax6.grid(True, alpha=0.3)
            
            plt.suptitle(f'Individual Cycle Analysis - {os.path.basename(cyclefolder)}', 
                        fontsize=15, fontweight='bold')
            plt.tight_layout()
            
            results['figures'].append(fig)
            results['data'].append(cycle_data)
    
    return results

# Example usage
settings = {
    'firstCrate': 0.1,
    'mincapacity': 5000,
    'xscale': 1000,
    'ylimithigh': 5500,
    'ylimitlow': 4000,
    'irscale': 2
}

# Mock data folder (replace with actual path)
# results = indiv_cyc_analysis(['./test_data'], settings)
print("✅ Individual Cycle Analysis function defined")

## 3. Linked Cycle Analysis <a id='link-cyc'></a>

**Purpose**: Link multiple cycle tests across different time periods

**Features**:
- CSV path configuration support
- Continuous cycle numbering across linked tests
- Cross-test comparison capability

In [ ]:
def link_cyc_indiv_analysis(tsv_files, settings):
    """
    Linked Cycle Individual Analysis
    
    Parameters:
        tsv_files: list of TSV files containing cyclepath and cyclename columns
        settings: dict with analysis settings
    
    Returns:
        dict with linked analysis results
    """
    results = {'figures': [], 'linked_data': []}
    
    for tsv_file in tsv_files:
        # Read TSV configuration
        cycle_paths = pd.read_csv(tsv_file, sep='\t', encoding='UTF-8', skiprows=1)
        
        all_data_folder = cycle_paths['cyclepath'].tolist()
        all_data_name = cycle_paths['cyclename'].tolist() if 'cyclename' in cycle_paths.columns else []
        
        fig, axes = plt.subplots(2, 3, figsize=(14, 8))
        
        cycle_max_list = [0] * 5  # Track max cycles per channel
        link_writerownum = [0] * 5  # Track row offsets
        
        for idx, cyclefolder in enumerate(all_data_folder):
            if not os.path.exists(cyclefolder):
                continue
                
            subfolders = [f.path for f in os.scandir(cyclefolder) if f.is_dir()]
            
            for chnl_idx, channel_folder in enumerate(subfolders):
                # Load cycle data with offset
                offset = link_writerownum[chnl_idx] + cycle_max_list[chnl_idx]
                
                # Mock linked data (adjust indices for continuity)
                cycles = np.arange(100) + offset
                capacity = np.random.rand(100) * settings['mincapacity']
                
                # Plot on shared axes
                label = all_data_name[idx] if idx < len(all_data_name) else f"Test {idx}"
                axes.flatten()[0].plot(cycles, capacity, 'o-', label=label, markersize=3)
                
                # Update trackers
                cycle_max_list[chnl_idx] = len(cycles)
        
        axes.flatten()[0].set_xlabel('Continuous Cycle Number')
        axes.flatten()[0].set_ylabel('Discharge Capacity (mAh)')
        axes.flatten()[0].legend()
        axes.flatten()[0].grid(True, alpha=0.3)
        
        plt.suptitle('Linked Cycle Analysis', fontsize=15, fontweight='bold')
        plt.tight_layout()
        
        results['figures'].append(fig)
    
    return results

print("✅ Linked Cycle Analysis function defined")

## 4. Step Profile Analysis <a id='step'></a>

**Purpose**: Analyze step-by-step charging profiles at specific cycle numbers

**Features**:
- Time-series analysis (V, SOC, C-rate, Temp)
- Multi-cycle overlay
- ECT format CSV export

In [ ]:
def step_profile_analysis(data_folders, cycle_numbers, settings):
    """
    Step Profile Analysis
    
    Parameters:
        data_folders: list of folder paths
        cycle_numbers: list of cycle numbers to analyze
        settings: dict with analysis settings
    
    Returns:
        dict with step profile results
    """
    results = {'figures': [], 'csv_data': []}
    
    for cyclefolder in data_folders:
        if not os.path.exists(cyclefolder):
            continue
            
        subfolders = [f.path for f in os.scandir(cyclefolder) if f.is_dir()]
        
        for channel_folder in subfolders:
            if 'Pattern' in channel_folder:
                continue
                
            fig, axes = plt.subplots(2, 3, figsize=(14, 10))
            
            for cyc_no in cycle_numbers:
                # Mock step profile data
                time_min = np.linspace(0, 180, 500)  # 3 hours
                voltage = 3.0 + (4.2 - 3.0) * (time_min / 180)  # Linear charge
                soc = time_min / 180
                crate = np.ones_like(time_min) * 0.5
                temp = 25 + np.random.randn(len(time_min)) * 2
                
                label = f"{cyc_no:04d}"
                
                # 6 panels: voltage (3x), SOC, C-rate, Temperature
                axes[0, 0].plot(time_min, voltage, label=label)
                axes[0, 1].plot(time_min, voltage, label=label)
                axes[0, 2].plot(time_min, voltage, label=label)
                axes[1, 0].plot(time_min, soc, label=label)
                axes[1, 1].plot(time_min, crate, label=label)
                axes[1, 2].plot(time_min, temp, label=label)
                
                # Prepare CSV export data
                csv_data = pd.DataFrame({
                    'TimeSec': time_min * 60,
                    'Voltage(V)': voltage.round(4),
                    'Current(A)': (crate * settings['mincapacity'] / 1000).round(4),
                    'Temp': temp.round(1)
                })
                results['csv_data'].append(csv_data)
            
            # Configure axes
            axes[0, 0].set_ylabel('Voltage (V)')
            axes[0, 1].set_ylabel('Voltage (V)')
            axes[0, 2].set_ylabel('Voltage (V)')
            axes[1, 0].set_ylabel('SOC')
            axes[1, 1].set_ylabel('C-rate')
            axes[1, 2].set_ylabel('Temperature (°C)')
            
            for ax in axes.flatten():
                ax.set_xlabel('Time (min)')
                ax.legend(loc='lower right')
                ax.grid(True, alpha=0.3)
            
            plt.suptitle('Step Profile Analysis', fontsize=15, fontweight='bold')
            plt.tight_layout()
            
            results['figures'].append(fig)
    
    return results

print("✅ Step Profile Analysis function defined")

## 5. Rate Capability Analysis <a id='rate'></a>

**Purpose**: Analyze battery performance at different C-rates

**Features**:
- Rate performance comparison
- Dual voltage views
- SOC tracking during rate testing

In [ ]:
def rate_capability_analysis(data_folders, cycle_numbers, settings):
    """
    Rate Capability Analysis
    
    Parameters:
        data_folders: list of folder paths
        cycle_numbers: list of cycle numbers (representing different C-rates)
        settings: dict with analysis settings
    
    Returns:
        dict with rate capability results
    """
    results = {'figures': [], 'rate_data': []}
    
    # Different C-rates for rate capability test
    c_rates = [0.1, 0.2, 0.5, 1.0, 2.0, 3.0]
    
    for cyclefolder in data_folders:
        if not os.path.exists(cyclefolder):
            continue
            
        subfolders = [f.path for f in os.scandir(cyclefolder) if f.is_dir()]
        
        for channel_folder in subfolders:
            if 'Pattern' in channel_folder:
                continue
                
            fig, axes = plt.subplots(2, 3, figsize=(14, 10))
            
            for idx, cyc_no in enumerate(cycle_numbers[:len(c_rates)]):
                c_rate = c_rates[idx]
                
                # Mock rate profile (higher C-rate = shorter time, lower capacity)
                time_min = np.linspace(0, 180/c_rate, 500)
                capacity_factor = 1 - (c_rate - 0.1) * 0.05  # Capacity fade at high rates
                voltage = 3.0 + (4.2 - 3.0) * (time_min / time_min[-1]) - (c_rate - 0.1) * 0.1
                soc = (time_min / time_min[-1]) * capacity_factor
                crate_profile = np.ones_like(time_min) * c_rate
                temp = 25 + c_rate * 5 + np.random.randn(len(time_min)) * 2
                
                label = f"{c_rate:.1f}C"
                
                # 6 panels: voltage (2x), C-rate (2x), SOC, Temperature
                axes[0, 0].plot(time_min, voltage, label=label)
                axes[0, 1].plot(time_min, crate_profile, label=label)
                axes[0, 2].plot(time_min, soc, label=label)
                axes[1, 0].plot(time_min, voltage, label=label)
                axes[1, 1].plot(time_min, crate_profile, label=label)
                axes[1, 2].plot(time_min, temp, label=label)
            
            # Configure axes
            axes[0, 0].set_ylabel('Voltage (V)')
            axes[0, 1].set_ylabel('C-rate')
            axes[0, 2].set_ylabel('SOC')
            axes[1, 0].set_ylabel('Voltage (V)')
            axes[1, 1].set_ylabel('C-rate')
            axes[1, 2].set_ylabel('Temperature (°C)')
            
            for ax in axes.flatten():
                ax.set_xlabel('Time (min)')
                ax.legend()
                ax.grid(True, alpha=0.3)
            
            plt.suptitle('Rate Capability Analysis', fontsize=15, fontweight='bold')
            plt.tight_layout()
            
            results['figures'].append(fig)
    
    return results

print("✅ Rate Capability Analysis function defined")

## 6. Charge Profile Analysis <a id='chg'></a>

**Purpose**: Detailed charge profile with dQ/dV and dV/dQ analysis

**Features**:
- Differential capacity (dQ/dV) curves
- Differential voltage (dV/dQ) curves
- Phase transition identification

In [ ]:
def charge_profile_analysis(data_folders, cycle_numbers, settings):
    """
    Charge Profile Analysis with dQ/dV and dV/dQ
    
    Parameters:
        data_folders: list of folder paths
        cycle_numbers: list of cycle numbers to analyze
        settings: dict with 'smoothdegree', 'dqscale', 'dvscale'
    
    Returns:
        dict with charge profile results
    """
    results = {'figures': [], 'differential_data': []}
    
    for cyclefolder in data_folders:
        if not os.path.exists(cyclefolder):
            continue
            
        subfolders = [f.path for f in os.scandir(cyclefolder) if f.is_dir()]
        
        for channel_folder in subfolders:
            if 'Pattern' in channel_folder:
                continue
                
            fig, axes = plt.subplots(2, 3, figsize=(14, 10))
            
            for cyc_no in cycle_numbers:
                # Mock charge profile
                soc = np.linspace(0, 1, 1000)
                voltage = 3.0 + 1.2 * soc + 0.2 * np.sin(soc * 10)  # Realistic V-SOC curve
                
                # Calculate differentials
                dqdv = np.gradient(soc, voltage)  # dQ/dV
                dvdq = np.gradient(voltage, soc)  # dV/dQ
                
                # Apply smoothing
                from scipy.ndimage import gaussian_filter1d
                smoothdegree = settings.get('smoothdegree', 5)
                dqdv_smooth = gaussian_filter1d(dqdv, smoothdegree)
                dvdq_smooth = gaussian_filter1d(dvdq, smoothdegree)
                
                crate = np.ones_like(soc) * 0.5
                temp = 25 + np.random.randn(len(soc)) * 2
                
                label = f"{cyc_no:04d}"
                
                # 6 panels: SOC-V (2x), dQ/dV, dV/dQ, C-rate, Temperature
                axes[0, 0].plot(soc, voltage, label=label)
                axes[0, 1].plot(voltage, dqdv_smooth, label=label)  # dQ/dV vs V
                axes[0, 2].plot(soc, voltage, label=label)
                axes[1, 0].plot(soc, dvdq_smooth, label=label)  # dV/dQ vs SOC
                axes[1, 1].plot(soc, crate, label=label)
                axes[1, 2].plot(soc, temp, label=label)
            
            # Configure axes
            axes[0, 0].set_xlabel('SOC')
            axes[0, 0].set_ylabel('Voltage (V)')
            axes[0, 1].set_xlabel('Voltage (V)')
            axes[0, 1].set_ylabel('dQ/dV')
            axes[0, 2].set_xlabel('SOC')
            axes[0, 2].set_ylabel('Voltage (V)')
            axes[1, 0].set_xlabel('SOC')
            axes[1, 0].set_ylabel('dV/dQ')
            axes[1, 1].set_xlabel('SOC')
            axes[1, 1].set_ylabel('C-rate')
            axes[1, 2].set_xlabel('SOC')
            axes[1, 2].set_ylabel('Temperature (°C)')
            
            for ax in axes.flatten():
                ax.legend()
                ax.grid(True, alpha=0.3)
            
            plt.suptitle('Charge Profile Analysis (dQ/dV, dV/dQ)', fontsize=15, fontweight='bold')
            plt.tight_layout()
            
            results['figures'].append(fig)
    
    return results

print("✅ Charge Profile Analysis function defined")

## 7. Discharge Profile Analysis <a id='dchg'></a>

**Purpose**: Detailed discharge profile with differential curves

**Features**:
- DOD (Depth of Discharge) based analysis
- Negative dQ/dV for discharge
- Mirror of charge profile analysis

In [ ]:
def discharge_profile_analysis(data_folders, cycle_numbers, settings):
    """
    Discharge Profile Analysis with dQ/dV and dV/dQ
    
    Parameters:
        data_folders: list of folder paths
        cycle_numbers: list of cycle numbers to analyze
        settings: dict with 'smoothdegree', 'dqscale', 'dvscale'
    
    Returns:
        dict with discharge profile results
    """
    results = {'figures': [], 'differential_data': []}
    
    for cyclefolder in data_folders:
        if not os.path.exists(cyclefolder):
            continue
            
        subfolders = [f.path for f in os.scandir(cyclefolder) if f.is_dir()]
        
        for channel_folder in subfolders:
            if 'Pattern' in channel_folder:
                continue
                
            fig, axes = plt.subplots(2, 3, figsize=(14, 10))
            
            for cyc_no in cycle_numbers:
                # Mock discharge profile (DOD = Depth of Discharge)
                dod = np.linspace(0, 1, 1000)
                voltage = 4.2 - 1.2 * dod - 0.2 * np.sin(dod * 10)  # Decreasing voltage
                
                # Calculate differentials (negative for discharge)
                dqdv = -np.gradient(dod, voltage)  # Negative dQ/dV
                dvdq = -np.gradient(voltage, dod)  # Negative dV/dQ
                
                # Apply smoothing
                from scipy.ndimage import gaussian_filter1d
                smoothdegree = settings.get('smoothdegree', 5)
                dqdv_smooth = gaussian_filter1d(dqdv, smoothdegree)
                dvdq_smooth = gaussian_filter1d(dvdq, smoothdegree)
                
                crate = np.ones_like(dod) * -0.5  # Negative for discharge
                temp = 25 + np.random.randn(len(dod)) * 2
                
                label = f"{cyc_no:04d}"
                
                # 6 panels: DOD-V (2x), dQ/dV, dV/dQ, C-rate, Temperature
                axes[0, 0].plot(dod, voltage, label=label)
                axes[0, 1].plot(dqdv_smooth, voltage, label=label)  # dQ/dV vs V
                axes[0, 2].plot(dod, voltage, label=label)
                axes[1, 0].plot(dod, dvdq_smooth, label=label)  # dV/dQ vs DOD
                axes[1, 1].plot(dod, crate, label=label)
                axes[1, 2].plot(dod, temp, label=label)
            
            # Configure axes
            axes[0, 0].set_xlabel('DOD')
            axes[0, 0].set_ylabel('Voltage (V)')
            axes[0, 1].set_xlabel('dQ/dV')
            axes[0, 1].set_ylabel('Voltage (V)')
            axes[0, 1].set_xlim(-5 * settings.get('dqscale', 1), 0.5 * settings.get('dqscale', 1))
            axes[0, 2].set_xlabel('DOD')
            axes[0, 2].set_ylabel('Voltage (V)')
            axes[1, 0].set_xlabel('DOD')
            axes[1, 0].set_ylabel('dV/dQ')
            axes[1, 1].set_xlabel('DOD')
            axes[1, 1].set_ylabel('C-rate')
            axes[1, 2].set_xlabel('DOD')
            axes[1, 2].set_ylabel('Temperature (°C)')
            
            for ax in axes.flatten():
                ax.legend()
                ax.grid(True, alpha=0.3)
            
            plt.suptitle('Discharge Profile Analysis (dQ/dV, dV/dQ)', fontsize=15, fontweight='bold')
            plt.tight_layout()
            
            results['figures'].append(fig)
    
    return results

print("✅ Discharge Profile Analysis function defined")

## 8. Continue Analysis (OCV/CCV) <a id='continue'></a>

**Purpose**: Continuous cycle analysis with OCV/CCV tracking

**Features**:
- Cycle range support (e.g., "1-100")
- OCV (Open Circuit Voltage) tracking
- CCV (Closed Circuit Voltage) tracking
- Dual Excel sheet export

In [ ]:
def continue_analysis(data_folders, cycle_range, settings):
    """
    Continue Analysis with OCV/CCV Tracking
    
    Parameters:
        data_folders: list of folder paths
        cycle_range: string like "1-100" or list of ranges
        settings: dict with analysis settings
    
    Returns:
        dict with continuous analysis results
    """
    results = {'figures': [], 'profile_data': [], 'ocv_ccv_data': []}
    
    # Parse cycle range
    if isinstance(cycle_range, str):
        cycle_range = [cycle_range]
    
    for cyclefolder in data_folders:
        if not os.path.exists(cyclefolder):
            continue
            
        subfolders = [f.path for f in os.scandir(cyclefolder) if f.is_dir()]
        
        for channel_folder in subfolders:
            for range_str in cycle_range:
                if '-' not in range_str:
                    continue
                    
                start_cyc, end_cyc = map(int, range_str.split('-'))
                
                fig, axes = plt.subplots(2, 3, figsize=(14, 10))
                
                # Mock continuous profile data
                total_time = (end_cyc - start_cyc + 1) * 180  # 3 hours per cycle
                time_min = np.linspace(0, total_time, 5000)
                
                # Continuous voltage profile with charge/discharge cycles
                voltage = 3.7 + 0.5 * np.sin(time_min / 180 * 2 * np.pi)
                crate = 0.5 * np.sin(time_min / 180 * 2 * np.pi)  # Alternating charge/discharge
                soc = 0.5 + 0.4 * np.sin(time_min / 180 * 2 * np.pi)
                temp = 25 + np.random.randn(len(time_min)) * 2
                
                # OCV/CCV at rest points (every 180 min)
                rest_times = np.arange(0, total_time, 180)
                ocv = 3.7 + 0.5 * np.sin(rest_times / 180 * 2 * np.pi)
                ccv = ocv - 0.05  # Slightly lower due to IR drop
                rest_soc = 0.5 + 0.4 * np.sin(rest_times / 180 * 2 * np.pi)
                
                # 6 panels: V-time (2x with OCV/CCV), C-rate, SOC, SOC-OCV/CCV, Temp
                axes[0, 0].plot(time_min, voltage, label='Voltage')
                axes[0, 1].plot(time_min, crate, label='C-rate')
                axes[0, 2].plot(time_min, soc, label='SOC')
                
                axes[1, 0].plot(time_min, voltage, label='Voltage')
                axes[1, 0].plot(rest_times, ocv, 'ro', markersize=4, label='OCV')
                axes[1, 0].plot(rest_times, ccv, 'go', markersize=4, label='CCV')
                
                axes[1, 1].plot(rest_soc, ocv, 'ro-', markersize=4, label='OCV')
                axes[1, 1].plot(rest_soc, ccv, 'go-', markersize=4, label='CCV')
                
                axes[1, 2].plot(time_min, temp, label='Temperature')
                
                # Configure axes
                axes[0, 0].set_ylabel('Voltage (V)')
                axes[0, 1].set_ylabel('C-rate')
                axes[0, 2].set_ylabel('SOC')
                axes[1, 0].set_ylabel('Voltage (V)')
                axes[1, 1].set_xlabel('SOC')
                axes[1, 1].set_ylabel('OCV/CCV (V)')
                axes[1, 2].set_ylabel('Temperature (°C)')
                
                for ax in axes.flatten():
                    ax.set_xlabel('Time (min)')
                    ax.legend()
                    ax.grid(True, alpha=0.3)
                
                # Fix x-label for SOC-OCV plot
                axes[1, 1].set_xlabel('SOC')
                
                plt.suptitle(f'Continue Analysis (Cycles {start_cyc}-{end_cyc})', 
                            fontsize=15, fontweight='bold')
                plt.tight_layout()
                
                # Prepare export data
                profile_df = pd.DataFrame({
                    'TimeSec': (time_min * 60).round(1),
                    'Voltage(V)': voltage.round(4),
                    'Current(A)': (crate * settings['mincapacity'] / 1000).round(4),
                    'OCV': np.interp(time_min, rest_times, ocv).round(4),
                    'CCV': np.interp(time_min, rest_times, ccv).round(4),
                    'Crate': crate.round(4),
                    'SOC': soc.round(4),
                    'Temp': temp.round(1)
                })
                
                ocv_ccv_df = pd.DataFrame({
                    'SOC': rest_soc.round(4),
                    'OCV': ocv.round(4),
                    'CCV': ccv.round(4)
                })
                
                results['figures'].append(fig)
                results['profile_data'].append(profile_df)
                results['ocv_ccv_data'].append(ocv_ccv_df)
    
    return results

print("✅ Continue Analysis function defined")

## 9. DCIR Analysis <a id='dcir'></a>

**Purpose**: DC Internal Resistance analysis across cycle ranges

**Features**:
- Multi-time DCIR (0.1s, 1.0s, 10.0s, 20.0s)
- RSS (Relaxation State of Charge) resistance
- SOC-based resistance mapping
- OCV estimation validation

In [ ]:
def dcir_analysis(data_folders, cycle_range, settings):
    """
    DCIR (DC Internal Resistance) Analysis
    
    Parameters:
        data_folders: list of folder paths
        cycle_range: string like "1-100" or list of ranges
        settings: dict with analysis settings
    
    Returns:
        dict with DCIR analysis results
    """
    results = {'figures': [], 'dcir_data': []}
    
    # Parse cycle range
    if isinstance(cycle_range, str):
        cycle_range = [cycle_range]
    
    for cyclefolder in data_folders:
        if not os.path.exists(cyclefolder):
            continue
            
        subfolders = [f.path for f in os.scandir(cyclefolder) if f.is_dir()]
        
        for channel_folder in subfolders:
            if 'Pattern' in channel_folder:
                continue
                
            for range_str in cycle_range:
                if '-' not in range_str:
                    continue
                    
                start_cyc, end_cyc = map(int, range_str.split('-'))
                
                fig, axes = plt.subplots(2, 2, figsize=(14, 8))
                
                # Mock DCIR data
                soc = np.linspace(0, 1, 11)  # 0%, 10%, ..., 100%
                ocv = 3.0 + 1.2 * soc  # Linear approximation
                r_ocv = ocv + np.random.randn(len(soc)) * 0.01  # Reverse OCV
                ccv = ocv - 0.05  # Closed circuit voltage
                
                # DCIR at different time intervals (resistance increases with time)
                dcir_01s = 20 + 30 * (1 - soc) + np.random.randn(len(soc)) * 2  # 0.1s
                dcir_10s = 25 + 35 * (1 - soc) + np.random.randn(len(soc)) * 2  # 1.0s
                dcir_100s = 30 + 40 * (1 - soc) + np.random.randn(len(soc)) * 2  # 10.0s
                dcir_200s = 35 + 45 * (1 - soc) + np.random.randn(len(soc)) * 2  # 20.0s
                rss = 40 + 50 * (1 - soc) + np.random.randn(len(soc)) * 2  # RSS
                
                # 4 panels: SOC-OCV/CCV, SOC-DCIR, V-SOC, OCV-DCIR
                # Panel 1: SOC vs OCV/rOCV/CCV
                axes[0, 0].plot(soc * 100, ocv, 'o-', label='OCV', markersize=6)
                axes[0, 0].plot(soc * 100, r_ocv, 's-', label='rOCV', markersize=6)
                axes[0, 0].plot(soc * 100, ccv, '^-', label='CCV', markersize=6)
                axes[0, 0].set_xlabel('SOC (%)')
                axes[0, 0].set_ylabel('Voltage (V)')
                axes[0, 0].set_title('OCV/CCV vs SOC')
                axes[0, 0].legend(loc='lower right')
                axes[0, 0].grid(True, alpha=0.3)
                
                # Panel 2: SOC vs DCIR (multiple time intervals)
                axes[0, 1].plot(soc * 100, dcir_01s, 'o-', label='0.1s DCIR', markersize=4)
                axes[0, 1].plot(soc * 100, dcir_10s, 's-', label='1.0s DCIR', markersize=4)
                axes[0, 1].plot(soc * 100, dcir_100s, '^-', label='10.0s DCIR', markersize=4)
                axes[0, 1].plot(soc * 100, dcir_200s, 'd-', label='20.0s DCIR', markersize=4)
                axes[0, 1].plot(soc * 100, rss, 'v-', label='RSS DCIR', markersize=4)
                axes[0, 1].set_xlabel('SOC (%)')
                axes[0, 1].set_ylabel('DCIR (mΩ)')
                axes[0, 1].set_title('DCIR vs SOC')
                axes[0, 1].legend(loc='upper right')
                axes[0, 1].grid(True, alpha=0.3)
                
                # Panel 3: Voltage vs SOC
                axes[1, 0].plot(ocv, soc * 100, 'o-', label='OCV', markersize=6)
                axes[1, 0].plot(ccv, soc * 100, '^-', label='CCV', markersize=6)
                axes[1, 0].set_xlabel('Voltage (V)')
                axes[1, 0].set_ylabel('SOC (%)')
                axes[1, 0].set_title('SOC vs Voltage')
                axes[1, 0].legend(loc='lower right')
                axes[1, 0].grid(True, alpha=0.3)
                
                # Panel 4: OCV vs DCIR
                axes[1, 1].plot(ocv, dcir_01s, 'o-', label='0.1s DCIR', markersize=4)
                axes[1, 1].plot(ocv, dcir_10s, 's-', label='1.0s DCIR', markersize=4)
                axes[1, 1].plot(ocv, dcir_100s, '^-', label='10.0s DCIR', markersize=4)
                axes[1, 1].plot(ocv, dcir_200s, 'd-', label='20.0s DCIR', markersize=4)
                axes[1, 1].plot(ocv, rss, 'v-', label='RSS DCIR', markersize=4)
                axes[1, 1].set_xlabel('OCV (V)')
                axes[1, 1].set_ylabel('DCIR (mΩ)')
                axes[1, 1].set_title('DCIR vs OCV')
                axes[1, 1].legend(loc='upper right')
                axes[1, 1].grid(True, alpha=0.3)
                
                plt.suptitle(f'DCIR Analysis (Cycles {start_cyc}-{end_cyc})', 
                            fontsize=15, fontweight='bold')
                plt.tight_layout()
                
                # Prepare export data
                dcir_df = pd.DataFrame({
                    'Capacity(mAh)': (soc * settings['mincapacity']).round(2),
                    'SOC': (soc * 100).round(2),
                    'OCV': ocv.round(4),
                    '0.1s_DCIR': dcir_01s.round(2),
                    '1.0s_DCIR': dcir_10s.round(2),
                    '10.0s_DCIR': dcir_100s.round(2),
                    '20.0s_DCIR': dcir_200s.round(2),
                    'RSS': rss.round(2),
                    'CCV': ccv.round(4)
                })
                
                results['figures'].append(fig)
                results['dcir_data'].append(dcir_df)
    
    return results

print("✅ DCIR Analysis function defined")

## Example Usage

Below are examples of how to use each function:

In [ ]:
# Example 1: Individual Cycle Analysis
settings = {
    'firstCrate': 0.1,
    'mincapacity': 5000,
    'xscale': 1000,
    'ylimithigh': 5500,
    'ylimitlow': 4000,
    'irscale': 2
}

# Uncomment to run with actual data
# results = indiv_cyc_analysis(['./test_data/cycle1'], settings)
# plt.show()

print("Example 1: Individual Cycle Analysis ready")

In [ ]:
# Example 2: Charge Profile with dQ/dV
profile_settings = {
    'mincapacity': 5000,
    'smoothdegree': 5,
    'dqscale': 1.0,
    'dvscale': 1.0
}

# Uncomment to run with actual data
# results = charge_profile_analysis(['./test_data/cycle1'], [1, 10, 50, 100], profile_settings)
# plt.show()

print("Example 2: Charge Profile Analysis ready")

In [ ]:
# Example 3: DCIR Analysis
dcir_settings = {
    'mincapacity': 5000,
    'firstCrate': 0.1
}

# Uncomment to run with actual data
# results = dcir_analysis(['./test_data/cycle1'], ['1-100'], dcir_settings)
# plt.show()

# Export DCIR data to Excel
# if results['dcir_data']:
#     results['dcir_data'][0].to_excel('dcir_output.xlsx', index=False)

print("Example 3: DCIR Analysis ready")

## Summary

This notebook implements all 8 button functions from the Battery Data Tool:

| Function | Key Output | Use Case |
|----------|-----------|----------|
| `indiv_cyc_analysis` | 6-panel cycle aging | Individual channel analysis |
| `link_cyc_indiv_analysis` | Linked cycle data | Long-term aging studies |
| `step_profile_analysis` | Time-series profiles | Charge progression |
| `rate_capability_analysis` | Rate performance | C-rate testing |
| `charge_profile_analysis` | dQ/dV, dV/dQ | Phase transitions |
| `discharge_profile_analysis` | Discharge differentials | Degradation analysis |
| `continue_analysis` | OCV/CCV tracking | Extended cycling |
| `dcir_analysis` | Multi-time DCIR | Impedance analysis |

**Note**: This notebook uses mock data for demonstration. Replace data loading sections with actual BatteryDataTool functions for real analysis.